In [1]:
### Importing the Network Data

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt

In [6]:
articles = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/articles.tsv", sep='\t', names = np.array(['article']), skiprows=12)

articles.head(10)

,article
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts
5,%E2%82%AC2_commemorative_coins
6,10th_century
7,11th_century
8,12th_century
9,13th_century


In [7]:
categories = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/categories.tsv", sep='\t', names = np.array(['article', 'category']), skiprows=13)

categories.head(10)

,article,category
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.History.British_History.British_Histor...
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.People.Historical_figures
2,%C3%85land,subject.Countries
3,%C3%85land,subject.Geography.European_Geography.European_...
4,%C3%89douard_Manet,subject.People.Artists
5,%C3%89ire,subject.Countries
6,%C3%89ire,subject.Geography.European_Geography.European_...
7,%C3%93engus_I_of_the_Picts,subject.History.British_History.British_Histor...
8,%C3%93engus_I_of_the_Picts,subject.People.Historical_figures
9,%E2%82%AC2_commemorative_coins,subject.Business_Studies.Currency


In [9]:
# The list of all links between articles.

links = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/links.tsv", sep='\t', names = np.array(['linkSource', 'linkTarget']), skiprows=12)

links.head(10)

,linkSource,linkTarget
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland
5,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Isle_of_Man
6,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Monarchy
7,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Orkney
8,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Picts
9,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Scotland


In [10]:
# Successful (i.e., finished) Wikispeedia paths.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# Articles in a path are separated by ";".
# Back clicks are represented as "<".
# Ratings are optionally given by the user after finishing the game and range from 1 ("easy") to 5 ("brutal").
# Missing ratings are represented as "NULL".

paths_finished = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/paths_finished.tsv", sep='\t', names = np.array(['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating']), skiprows=16)

paths_finished.head(10)

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
5,5295bca242be81fe,1372890414,110,14th_century;Europe;North_America;United_State...,NaN
6,36dabfa133b20e3c,1249525912,112,14th_century;China;Gunpowder;Fire,2.0
7,20418ff4797f96be,1229188046,139,14th_century;Time;Isaac_Newton;Light;Color;Rai...,1.0
8,08888b1b428dd90e,1232241510,74,14th_century;Time;Light;Rainbow,3.0
9,08888b1b428dd90e,1232241601,167,14th_century;15th_century;Plato;Nature;Ultravi...,NaN


In [8]:
# Unsuccessful (i.e., unfinished) Wikispeedia paths.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# Articles in a path are separated by ";".
# Back clicks are represented as "<".
# There are two types of quitting:
# (1) "timeout" means that no click was made for 30 minutes;
# (2) "restart" means that the user started a new game without finishing the current one.

paths_unfinished = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/paths_unfinished.tsv", sep='\t', names = np.array(['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type']), skiprows=17)

paths_unfinished.head(10)

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
5,131600803df4895e,1297100557,2009,Agriculture;History_of_the_world;China;Yangtze...,Grand_Canal_of_China,timeout
6,486bb79910fe9dd2,1297101660,1932,Mind;Christianity;Europe;Renaissance;Ancient_G...,Scouting,timeout
7,6d136e371e42474f,1297102070,175,4-2-0;United_States;Agriculture;Sugar;Australia,Cane_Toad,restart
8,6d136e371e42474f,1297102100,19,Logic,Moon_landing,restart
9,6d136e371e42474f,1297104110,2006,Logic;Computer_science;Science;Physical_scienc...,Moon_landing,timeout


In [11]:
# The shortest-path distances between all pairs of articles, computed using the Floyd-Warshall algorithm.
# FORMAT:
# One row per article (the "source" of the shortest paths), in the same order as in articles.tsv.
# Each row contains the distances from the source to all articles (the "targets" of the shortest paths),
# again in the order of articles.tsv.
# The shortest-path distance is represented as a single digit, with no separators between values. This
# is possible because the longest shortest path happens to be of length 9.
# An underscore ("_") is used to indicate that the target cannot be reached from the source.

shortest_path = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/TF3/Wikispeedia/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt", sep='\t', names = np.array(['path']), skiprows=17)

shortest_path.head(10)

,path
0,0_____33333325634333435_2433544334_3_422343544...
1,_0____22222325623232424_2422544324_3_312242544...
2,__0___33222425623232324_2333444433_3_422343434...
3,___0__33333325634233334_2433434333_2_423343433...
4,____0_22323335633332435_2433545434_3_423343544...
5,_____032223324523222324_2332544333_3_422333544...
6,______01222324523232423_2222544334_3_422232544...
7,______10111414512131334_1311544323_2_311131544...
8,______11011414512131324_1311544323_2_311131544...
9,______22101325623232434_2422545434_3_422242544...
